# Section 3: Clustering the Toronto Neighborhoods

In [ ]:
!pip install BeautifulSoup4
!pip install requests
!pip install lxml
!pip install html5lib

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
# import some more libraries
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [ ]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

In [ ]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

In [ ]:
df_toronto=pd.read_csv('df_toronto.csv')
df_toronto.drop(['Unnamed: 0'],axis=1, inplace=True)
df_toronto.head()

In [ ]:
## markers for the different neighborhoods

for lat, lng, borough, neighborhood in zip(
        df_toronto['Latitude'], 
        df_toronto['Longitude'], 
        df_toronto['Borough'], 
        df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [ ]:
## Use only boroughs that contain "Toronto"
df_toronto_only = df_toronto[df_toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto_only.head()


In [ ]:
##Mapping

map_toronto_only = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(
        df_toronto_only['Latitude'], 
        df_toronto_only['Longitude'], 
        df_toronto_only['Borough'], 
        df_toronto_only['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_only)  

map_toronto_only

In [ ]:
CLIENT_ID = 'V5EQ4YQMFHPTOVZVDRC1M5EJLTZ4XOYW4BPTOU20XYAZ0O5T'
CLIENT_SECRET = 'YERRGFUUDS4XORVZ3MLZI1XK5NNQ23ZK0V4GUJF0TVD5XKPJ'
VERSION = '20200602'

In [ ]:
neighborhood_name = df_toronto_only.loc[0, 'Neighborhood']
print(f"The first neighborhood's name is '{neighborhood_name}'.")

In [ ]:
neighborhood_latitude = df_toronto_only.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto_only.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

In [ ]:
## Top 75 venues in 500 meter radius of Regent Park

LIMIT = 75 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# get the result to a json file
results = requests.get(url).json()

In [ ]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
## Create Dataframe with JSON file

venues = results['response']['groups'][0]['items']
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
df_toronto_only_venues = getNearbyVenues(names=df_toronto_only['Neighborhood'],
                                   latitudes=df_toronto_only['Latitude'],
                                   longitudes=df_toronto_only['Longitude'])

In [ ]:
df_toronto_only_venues.head()

In [ ]:
## Drilling Down into Neighborhoods

# one hot encoding
toronto_only_onehot = pd.get_dummies(df_toronto_only_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_only_onehot['Neighborhood'] = df_toronto_only_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_only_onehot.columns[-1]] + list(toronto_only_onehot.columns[:-1])
toronto_only_onehot = toronto_only_onehot[fixed_columns]

toronto_only_onehot.head()

In [ ]:
#group by neighborhood an calculate average occurance

toronto_only_grouped = toronto_only_onehot.groupby('Neighborhood').mean().reset_index()

toronto_only_grouped.head()

In [ ]:
## the 10 Most Common Venues

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_only_grouped['Neighborhood']

for ind in np.arange(toronto_only_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_only_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
## CLustering the Neighborhoods

kclusters = 5

toronto_only_grouped_clustering = toronto_only_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_only_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_only_merged = df_toronto_only


toronto_only_merged = toronto_only_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_only_merged.head() # check the last columns!

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        toronto_only_merged['Latitude'], 
        toronto_only_merged['Longitude'], 
        toronto_only_merged['Neighborhood'], 
        toronto_only_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
## The 5 Clusters

In [ ]:
toronto_only_merged.loc[toronto_only_merged['Cluster Labels'] == 0, toronto_only_merged.columns[[1] + list(range(5, toronto_only_merged.shape[1]))]]

In [ ]:
toronto_only_merged.loc[toronto_only_merged['Cluster Labels'] == 1, toronto_only_merged.columns[[1] + list(range(5, toronto_only_merged.shape[1]))]]

In [ ]:
toronto_only_merged.loc[toronto_only_merged['Cluster Labels'] == 2, toronto_only_merged.columns[[1] + list(range(5, toronto_only_merged.shape[1]))]]

In [ ]:
toronto_only_merged.loc[toronto_only_merged['Cluster Labels'] == 3, toronto_only_merged.columns[[1] + list(range(5, toronto_only_merged.shape[1]))]]

In [ ]:
toronto_only_merged.loc[toronto_only_merged['Cluster Labels'] == 4, toronto_only_merged.columns[[1] + list(range(5, toronto_only_merged.shape[1]))]]